In [ ]:
"""総合演習2 マクロ系シミュレーション 第1回 例題3

2次元定常拡散方程式のシミュレーション(有限差分法)
"""

import sys
from typing import Final

import matplotlib.pyplot as plt
import numpy as np

# ========== パラメータ ==========
LX: Final[float] = 1            # 部屋のx方向の長さ(m)
LY: Final[float] = LX           # 部屋のy方向の長さ(m)
D: Final[float] = 1             # 二酸化炭素の拡散係数(m^2/s)
C_OUT: Final[float] = 0         # 外気の二酸化炭素濃度(ppm)

NX: Final[int] = 31             # x方向の格子点数
NY: Final[int] = NX             # y方向の格子点数

OMEGA_SOR: Final[float] = 1.8   # SOR法の加速パラメータ(0<OMEGA_SOR<2)
TOL: Final[float] = 10**(-5)    # SOR法の許容誤差(相対誤差)
MAX_ITER: Final[int] = 1000     # SOR法の最大反復回数
# ==============================

# float型のNumPy配列の型エイリアス
FloatArray = np.typing.NDArray[np.float64]


def func_src(x: float, y: float, lx: float, ly: float) -> float:
    """(x,y)における二酸化炭素の生成項を計算する

    Parameters
    ----------
    x : float
        x座標
    y : float
        y座標
    lx : float
        部屋のx方向の長さ
    ly : float
        部屋のy方向の長さ

    Returns
    -------
    float
        (x,y)における二酸化炭素の生成項
    """

    return 1


class Variable:
    """格子点上の物理量を扱うクラス

    Attributes
    ----------
    value : FloatArray
        格子点上の物理量の値
    __name : str
        'grid_x', 'grid_y', 'co2', 'src' のいずれか

    Notes
    -----
    インスタンスを生成する前に, set_class_variablesクラスメソッドを実行する必要あり
    Variableクラスの外で使用しない属性は, 変数名の最初にダンダー(__)をつけて隠蔽している
    """

    __lx: float
    __ly: float
    __nx: int
    __ny: int
    __dx: float
    __dy: float

    # set_class_variablesクラスメソッドを実行したかどうかを判定する真偽値
    __flag: bool = False

    @classmethod
    def set_class_variables(cls, lx: float, ly: float, nx: int, ny: int) -> None:
        """Variableクラスのクラス変数を設定する

        Parameters
        ----------
        lx : float
            部屋のx方向の長さ
        ly : float
            部屋のy方向の長さ
        nx : int
            x方向の格子点数
        ny : int
            y方向の格子点数
        """

        cls.__lx = lx
        cls.__ly = ly
        cls.__nx = nx
        cls.__ny = ny
        cls.__dx = lx / (nx-1)
        cls.__dy = ly / (ny-1)
        cls.__flag = True

    def __init__(self, name: str) -> None:
        """Variableクラスのイニシャライザ

        Parameters
        ----------
        name : str
            'grid_x', 'grid_y', 'co2', 'src' のいずれか
        """

        if not Variable.__flag:
            print('[ERROR] set_class_variablesクラスメソッドが実行されていません')
            sys.exit()

        if name not in ['grid_x', 'grid_y', 'co2', 'src']:
            print('[ERROR] Variableクラスのイニシャライザの引数が不適切です')
            sys.exit()
        self.__name: str = name

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        self.value: FloatArray = np.empty((nx, ny), dtype=np.float64)

        lx: float = Variable.__lx
        ly: float = Variable.__ly
        dx: float = Variable.__dx
        dy: float = Variable.__dy
        x: float
        y: float
        for ix in range(nx):
            x = dx * ix
            for iy in range(ny):
                y = dy * iy
                self.value[ix, iy] = self.__set_initial_condition(x, y, lx, ly)

    def __set_initial_condition(self, x: float, y: float, lx: float, ly: float) -> float:
        """(x,y)における(初期条件の)値を設定する

        Parameters
        ----------
        x : float
            x座標
        y : float
            y座標
        lx : float
            部屋のx方向の長さ
        ly : float
            部屋のy方向の長さ

        Returns
        -------
        float
            (x,y)における(初期条件の)値
        """

        if self.__name == 'grid_x':
            return x
        elif self.__name == 'grid_y':
            return y
        elif self.__name == 'co2':
            return 1   # SOR法の初期値
        elif self.__name == 'src':
            return func_src(x, y, lx, ly)

    def set_boundary_condition(self, source: FloatArray, c_out: float) -> None:
        """二酸化炭素濃度の境界条件を設定する

        Parameters
        ----------
        source : FloatArray
            Poisson方程式のソース項
        c_out : float
            外気の二酸化炭素濃度
        """

        if self.__name == 'co2':

            nx: int = Variable.__nx
            ny: int = Variable.__ny
            dx: float = Variable.__dx
            dy: float = Variable.__dy
            denominator: float = 2/(dx**2) + 2/(dy**2)

            # 上端
            self.value[:, -1] = c_out

            # 左端
            for iy in range(ny-2, 0, -1):
                self.value[0, iy] = (
                    2*self.value[1, iy] / (dx**2)
                    + (self.value[0, iy+1] + self.value[0, iy-1]) / (dy**2)
                    - source[0, iy]
                ) / denominator

            # 左下端
            self.value[0, 0] = (
                2*self.value[1, 0] / (dx**2) + 2*self.value[0, 1] / (dy**2)
                - source[0, 0]
            ) / denominator

            # 下端
            for ix in range(1, nx-1):
                self.value[ix, 0] = (
                    (self.value[ix+1, 0] + self.value[ix-1, 0]) / (dx**2)
                    + 2*self.value[ix, 1] / (dy**2)
                    - source[ix, 0]
                ) / denominator

            # 右下端
            self.value[-1, 0] = (
                2*self.value[-2, 0] / (dx**2) + 2*self.value[-1, 1] / (dy**2)
                - source[-1, 0]
            ) / denominator

            # 右端
            for iy in range(1, ny-1):
                self.value[-1, iy] = (
                    2*self.value[-2, iy] / (dx**2)
                    + (self.value[-1, iy+1] + self.value[-1, iy-1]) / (dy**2)
                    - source[-1, iy]
                ) / denominator

    def poisson_solver(self, source: FloatArray, omega_sor: float, tol: float, max_iter: int, c_out: float) -> None:
        """そのインスタンスを未知関数とするPoisson方程式を解く(SOR法)

        SOR(Successive Over-Relaxation, 逐次加速緩和)法:
            Gauss-Seidel法における値の更新を過剰に行うことで反復回数を減らし, 解への収束を速める

        Parameters
        ----------
        source : FloatArray
            Poisson方程式のソース項
        omega_sor : float
            SOR法の加速パラメータ(0<omega_sor<2)
        tol : float
            SOR法の許容誤差(相対誤差)
        max_iter : int
            SOR法の最大反復回数
        c_out : float
            外気の二酸化炭素濃度
        """

        nx: int = Variable.__nx
        ny: int = Variable.__ny
        dx: float = Variable.__dx
        dy: float = Variable.__dy
        denominator: float = 2/(dx**2) + 2/(dy**2)

        max_err: float
        old: float
        new: float
        err: float
        for _ in range(max_iter):
            max_err = 0
            
            for ix in range(1, nx-1):
                for iy in range(1, ny-1):
                    old = self.value[ix, iy]
                    new = (
                        (self.value[ix+1, iy] + self.value[ix-1, iy]) / (dx**2)
                        + (self.value[ix, iy+1] + self.value[ix, iy-1]) / (dy**2)
                        - source[ix, iy]
                    ) / denominator
                    self.value[ix, iy] = (1-omega_sor) * old + omega_sor * new

                    if new != 0:
                        err = abs((new - old) / new)
                        max_err = max(max_err, err)

            self.set_boundary_condition(source, c_out)
            if max_err < tol:
                break


if __name__ == '__main__':

    Variable.set_class_variables(LX, LY, NX, NY)

    # Variableクラスのインスタンスとして変数を準備
    grid_x: Variable = Variable('grid_x')
    grid_y: Variable = Variable('grid_y')
    co2: Variable = Variable('co2')
    src: Variable = Variable('src')

    # SOR法で連立方程式を解く
    co2.poisson_solver(-src.value / D, OMEGA_SOR, TOL, MAX_ITER, C_OUT)

    figure, axes = plt.subplots(1, 2, figsize=(10, 5))

    # co2の等値線図
    vmin: float = C_OUT
    vmax: float = np.max(co2.value)
    im1 = axes[0].contourf(grid_x.value, grid_y.value, co2.value, vmin=vmin, vmax=vmax, levels=20, cmap='jet')
    axes[0].contour(im1, colors='k', linewidths=0.5)
    figure.colorbar(im1, ax=axes[0])
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_title('CO2')
    axes[0].set_aspect('equal')

    # srcの等値線図
    vmin = 0
    vmax = np.max(src.value)
    im2 = axes[1].contourf(grid_x.value, grid_y.value, src.value, vmin=vmin, vmax=vmax, levels=20, cmap='Greens')
    axes[1].contour(im2, colors='k', linewidths=0.5)
    figure.colorbar(im2, ax=axes[1])
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    axes[1].set_title('S0')
    axes[1].set_aspect('equal') 

    figure.tight_layout()
